<a href="https://colab.research.google.com/github/gustavesg/TCC/blob/main/3_Modelo_Risco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Importações bibliotecas e base**




In [ ]:
# Importar bibliotecas necessárias + drive

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import kstest
from scipy.stats import mannwhitneyu
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,precision_score,recall_score,log_loss
from sklearn.metrics import confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importação base de treino e teste
df_treino = pd.read_csv('/content/drive/MyDrive/TCC/1. Base de Dados/dados_finais_treino/base_no_normalizada_treino')
df_teste = pd.read_csv('/content/drive/MyDrive/TCC/1. Base de Dados/dados_finais_teste/base_no_normalizada_teste')

### **Nova variável: Comprometimento Renda**

In [ ]:
############################################# BASE TREINO ########################################################

#variável de renda mensal média
df_treino['renda_mensal'] = df_treino['annual_inc']/12


#renda futura (inflação média dos EUA no período considerado)
renda_futura = df_treino['annual_inc']

#indicador de capacidade de pagamento do empréstimo
comprometimento = ((1 + 0.01*df_treino['int_rate'])**60)*df_treino['loan_amnt']/60

#index
df_treino['index'] = comprometimento/renda_futura

############################################# BASE TESTE ########################################################

#variável de renda mensal média
df_teste['renda_mensal'] = df_teste['annual_inc']/12


#renda futura (inflação média dos EUA no período considerado)
renda_futura = df_teste['annual_inc']

#indicador de capacidade de pagamento do empréstimo
comprometimento = ((1 + 0.01*df_teste['int_rate'])**60)*df_teste['loan_amnt']/60

#index
df_teste['index'] = comprometimento/renda_futura

#################################################################################################################


### **Hipótese de igualdade de média entre bons e maus pagadores**

In [ ]:
#Aplicamos o teste de Mann-Whitney U para verificar a hipótese de que a média das taxas pra cada grupo são iguais

#nivel de significancia
alpha = 0.05

for column in df_treino.columns:

    #taxas para bons e maus pagadores
    values_good = df_treino[df_treino['loan_status'] == 1][column].dropna().to_numpy()
    values_bad = df_treino[df_treino['loan_status'] == 0][column].dropna().to_numpy()


    # Realizando o teste de Mann-Whitney U
    stat, p_value = mannwhitneyu(values_good, values_bad)

    print(104*'=')
    print(f'Análise pra coluna : {column}')
    print(104*'=')


    print('Estatística do teste:', stat)
    print('Valor-p:', p_value)

    # Interpretação do resultado
    if p_value < alpha:
        print('Rejeita a hipótese nula: as populações têm médias diferentes')
    else:
        print('Falha em rejeitar a hipótese nula: não há evidências suficientes para dizer que as populações têm médias diferentes')

    print(104*'=')


Análise pra coluna : Unnamed: 0
Estatística do teste: 0.0
Valor-p: 0.0
Rejeita a hipótese nula: as populações têm médias diferentes
Análise pra coluna : int_rate
Estatística do teste: 8157330778.5
Valor-p: 0.0
Rejeita a hipótese nula: as populações têm médias diferentes
Análise pra coluna : dti
Estatística do teste: 10706836515.0
Valor-p: 0.0
Rejeita a hipótese nula: as populações têm médias diferentes
Análise pra coluna : bc_open_to_buy
Estatística do teste: 13906000951.5
Valor-p: 0.0
Rejeita a hipótese nula: as populações têm médias diferentes
Análise pra coluna : avg_cur_bal
Estatística do teste: 13778693230.5
Valor-p: 0.0
Rejeita a hipótese nula: as populações têm médias diferentes
Análise pra coluna : revol_bal
Estatística do teste: 12494689859.5
Valor-p: 4.982102550326744e-05
Rejeita a hipótese nula: as populações têm médias diferentes
Análise pra coluna : annual_inc
Estatística do teste: 13552492352.0
Valor-p: 3.038504356932678e-298
Rejeita a hipótese nula: as populações têm méd

In [ ]:
df_treino[(df_treino['loan_status'] == 0)]['index'].median()

17.811056565230267

In [ ]:
df_treino[(df_treino['loan_status'] == 1)]['index'].median()

3.343824052604956

### **Parâmetro de Permissividade**

In [ ]:
permissividade = 0.50

### **Ponto de corte : Index**

In [ ]:
soma_index = 0

for j in range(1,1000):

  amostra_maus_pagadores = df_treino[(df_treino['loan_status'] == 0)].sample(n = 1000, random_state = 62349605)
  amostra_bons_pagadores = df_treino[(df_treino['loan_status'] == 1)].sample(n = 1000, random_state = 62349605)


  #define o corte inicial 15
  index = 100

  contagem_erro_tipo_1 = 1
  contagem_erro_tipo_2 = 0




  while contagem_erro_tipo_1/(contagem_erro_tipo_1 + contagem_erro_tipo_2)  > permissividade:

    contagem_erro_tipo_1 = amostra_maus_pagadores[amostra_maus_pagadores['index'] < index].shape[0]
    contagem_erro_tipo_2 = amostra_bons_pagadores[amostra_bons_pagadores['index'] >= index].shape[0]


    index = index - 0.5


  soma_index += index


print('index_medio = ', soma_index/1000)

index_medio =  6.993


In [ ]:
ponto_corte_index =  soma_index/1000

### **Modelo: fase I**

In [ ]:
#verdadeiros positivos
VP =  df_teste[(df_teste['loan_status'] == 1) & (df_teste['index'] < ponto_corte_index)].shape[0]

#verdadeiros negativos
VN =  df_teste[(df_teste['loan_status'] == 0) & (df_teste['index'] >= ponto_corte_index)].shape[0]

#falsos positivos
FP = df_teste[(df_teste['loan_status'] == 0) & (df_teste['index'] < ponto_corte_index)].shape[0]

#falsos negativos
FN = df_teste[(df_teste['loan_status'] == 1) & (df_teste['index'] >= ponto_corte_index)].shape[0]

#total previsões
T = VP+VN+FP+FN

#print acurácia
print(f'Acurácia = {(VP+VN)/T}')

#print especificidade
print(f'Especificidade = {VN/(VN+FP)}')

#print precisão
print(f'Precisão = {VP/(VP+FP)}')


Acurácia = 0.6333033693933852
Especificidade = 0.6483412606718982
Precisão = 0.6216848799857952


### **Ponto de Corte: DTI**

In [ ]:
sub_df = df_treino[df_treino['index'] < ponto_corte_index]

soma_corte_dti = 0

for j in range(1,1000):

  amostra_maus_pagadores = sub_df[(sub_df['loan_status'] == 0)].sample(n = 1000, random_state = 62349605)
  amostra_bons_pagadores = sub_df[(sub_df['loan_status'] == 1)].sample(n = 1000, random_state = 62349605)


  #define o corte inicial 15
  dti = 0

  contagem_erro_tipo_1 = 1
  contagem_erro_tipo_2 = 0

  while contagem_erro_tipo_1/(contagem_erro_tipo_1 + contagem_erro_tipo_2)  > permissividade:

    contagem_erro_tipo_1 = amostra_maus_pagadores[amostra_maus_pagadores['dti'] > dti].shape[0]
    contagem_erro_tipo_2 = amostra_bons_pagadores[amostra_bons_pagadores['dti'] <= dti].shape[0]

    dti = dti + 0.5

    #print(soma_corte_dti/j,contagem_erro_tipo_1,contagem_erro_tipo_2)


  soma_corte_dti += dti


print('dti_medio = ', soma_corte_dti/1000)

dti_medio =  17.982


In [ ]:
ponto_corte_dti = soma_corte_dti/1000

### **Modelo: fase II**

In [ ]:
#verdadeiros positivos
VP =  df_teste[(df_teste['loan_status'] == 1) & (df_teste['index'] < ponto_corte_index) & (df_teste['dti'] < 20.979)].shape[0]

#verdadeiros negativos
VN =  df_teste[(df_teste['loan_status'] == 0) & (df_teste['index'] >= ponto_corte_index)].shape[0] + df_teste[(df_teste['loan_status'] == 0) & (df_teste['index'] < ponto_corte_index) & (df_teste['dti'] >= ponto_corte_dti)].shape[0]

#falsos positivos
FP = df_teste[(df_teste['loan_status'] == 0) & (df_teste['index'] < ponto_corte_index) & (df_teste['dti'] < 20.979)].shape[0]

#falsos negativos
FN = df_teste[(df_teste['loan_status'] == 1) & (df_teste['index'] >= ponto_corte_index)].shape[0] + df_teste[(df_teste['loan_status'] == 1) & (df_teste['index'] < ponto_corte_index) & (df_teste['dti'] >= ponto_corte_dti)].shape[0]

#total previsões
T = VP + VN + FP + FN


#print acurácia
print(f'Acurácia = {(VP+VN)/T}')

#print especificidade
print(f'Especificidade = {VN/(VN+FP)}')

#print precisão
print(f'Precisão = {VP/(VP+FP)}')



Acurácia = 0.5989080064905639
Especificidade = 0.7909543699493319
Precisão = 0.6481955650031402


### **Ponto de Corte : avg_cur_bal**

In [ ]:
sub_sub_df = sub_df[sub_df['dti'] < ponto_corte_dti]

soma_corte_avg_cur_bal = 0

for j in range(1,1000):

  amostra_maus_pagadores = sub_sub_df[(sub_sub_df['loan_status'] == 0)].sample(n = 1000, random_state = 62349605)
  amostra_bons_pagadores = sub_sub_df[(sub_sub_df['loan_status'] == 1)].sample(n = 1000, random_state = 62349605)


  #define o corte inicial 8000
  avg_cur_bal  = 0

  contagem_erro_tipo_1 = 1
  contagem_erro_tipo_2 = 0

  while contagem_erro_tipo_1/(contagem_erro_tipo_1 + contagem_erro_tipo_2)  > permissividade:


    contagem_erro_tipo_1 = amostra_maus_pagadores[amostra_maus_pagadores['avg_cur_bal'] >  avg_cur_bal].shape[0]
    contagem_erro_tipo_2 = amostra_bons_pagadores[amostra_bons_pagadores['avg_cur_bal'] <= avg_cur_bal].shape[0]

    avg_cur_bal = avg_cur_bal + 500

    #print(contagem_erro_tipo_1,contagem_erro_tipo_2)


  soma_corte_avg_cur_bal += avg_cur_bal


print('avg_cur_bal_medio = ', soma_corte_avg_cur_bal/1000)

avg_cur_bal_medio =  7492.5


In [ ]:
ponto_corte_avg_cur_bal = soma_corte_avg_cur_bal/1000

### **Modelo : fase III**

In [ ]:
#verdadeiros positivos
VP =  df_teste[(df_teste['loan_status'] == 1) & (df_teste['index'] < ponto_corte_index) & (df_teste['dti'] < ponto_corte_dti) & (df_teste['avg_cur_bal'] > ponto_corte_avg_cur_bal)].shape[0]

#verdadeiros negativos
VN =  df_teste[(df_teste['loan_status'] == 0) & (df_teste['index'] >= ponto_corte_index)].shape[0] + df_teste[(df_teste['loan_status'] == 0) & (df_teste['index'] < ponto_corte_index) & (df_teste['dti'] >= ponto_corte_dti)].shape[0] + df_teste[(df_teste['loan_status'] == 0) & (df_teste['index'] < ponto_corte_index) & (df_teste['dti'] < ponto_corte_dti) & (df_teste['avg_cur_bal'] <= ponto_corte_avg_cur_bal)].shape[0]

#falsos positivos
FP = df_teste[(df_teste['loan_status'] == 0) & (df_teste['index'] < ponto_corte_index) & (df_teste['dti'] < ponto_corte_dti) & (df_teste['avg_cur_bal'] > ponto_corte_avg_cur_bal)].shape[0]

#falsos negativos
FN = df_teste[(df_teste['loan_status'] == 1) & (df_teste['index'] >= ponto_corte_index)].shape[0] + df_teste[(df_teste['loan_status'] == 1) & (df_teste['index'] < ponto_corte_index) & (df_teste['dti'] >= ponto_corte_dti)].shape[0] + df_teste[(df_teste['loan_status'] == 1) & (df_teste['index'] < ponto_corte_index) & (df_teste['dti'] < ponto_corte_dti) & (df_teste['avg_cur_bal'] <= ponto_corte_avg_cur_bal)].shape[0]

#total previsões
T = VP + VN + FP + FN

#acurácia
acuracia = (VP+VN)/T

#precisão
precisao = VP/(VP+FP)

#sensibilidade
recall = VP/(FN + VP)



### **Função de classificação do mutuário**

In [ ]:
def classifica_modelo(df: pd.DataFrame):

  if df['index'] >= ponto_corte_index: return 0

  else:

      if df['dti'] >= ponto_corte_dti: return 0

      else:

          if df['avg_cur_bal'] <= ponto_corte_avg_cur_bal: return 0

          else: return 1

df_teste['y_pred'] = df_teste.apply(classifica_modelo, axis=1)

In [ ]:
#calcula f1 score ponderado

#defina os pesos para cada classe
pesos = {0: 1- alpha, 1: alpha}

# Calcule o F1 Score para cada classe
f1_class_0 = f1_score(df_teste['loan_status'], df_teste['y_pred'], pos_label=0, average='binary')
f1_class_1 = f1_score(df_teste['loan_status'], df_teste['y_pred'], pos_label=1, average='binary')

# Calcule o F1 Score ponderado manualmente
f1_ponderado = (f1_class_0 * pesos[0] + f1_class_1 * pesos[1]) / sum(pesos.values())

print("F1 Score Ponderado: ", f1_ponderado)
print("Acurácia: ", acuracia)
print("Precisão: ", precisao)
print("Sensibilidade: ", recall)

F1 Score Ponderado:  0.6673175475141693
Acurácia:  0.563559716065838
Precisão:  0.7003891050583657
Sensibilidade:  0.1702004904081927


In [ ]:
round(100*(VP+FP)/T,2)

11.75

In [ ]:
sum(df_teste['y_pred'])/df_teste['y_pred'].shape[0]

0.11750255726728867